In [1]:
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
from circuits.format import Bits, format_msg, bitfun
from circuits.core import Signal
from circuits.examples.sha3 import sha3
from circuits.compile import compile_from_example
from circuits.torch_mlp import StepMLP
from msc_project.models.ga_compatible_stepml import create_gacompatible_stepmlp_from_message

In [2]:
mlp_template, input_tensor, output_tensor = create_gacompatible_stepmlp_from_message("Spaghetti", n_rounds=3)

In [3]:
num_params = sum(p.numel() for p in mlp_template.parameters())
print(f"Number of parameters in the MLP: {num_params}")

Number of parameters in the MLP: 260153576
